## 

In [ ]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from collections import Counter
from typing import List

In [ ]:
# TextRank
from gensim.summarization.summarizer import summarize

In [ ]:
# kiwi: Tokenizer
from kiwipiepy import Kiwi
# keyBert
from keybert import KeyBERT
from transformers import BertModel

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
import sys
sys.path.clear()
sys.path.append('/'.join(os.getcwd().split("\\")[:-1]))

## data 전처리

### Tokenize

In [ ]:
kiwi = Kiwi()

In [ ]:
# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    # print(result)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
            results.append(token)
    return results

### 불용어 제거

In [ ]:
stopwords = []

In [ ]:
from crawling import config
#  from src.crawling.config import connect, close

conn = config.connect()
curs = conn.cursor()

channel_sql = """select name from channel group by name"""
curs.execute(channel_sql)
channel_info = curs.fetchall()

channels = []
for channel in channel_info:
    channels.extend(channel)
print(channels)

config.close()

stopwords.extend(channels)

## keyword 추출

In [ ]:
def keyword_extractor(bert:str, documents:List[str]) -> (List[Tuple[str, float]] | List[List[Tuple[str, float]]]):
  model = BertModel.from_pretrained(bert)
  model = model.cuda()
  kw_model = KeyBERT(model)
  keywords = kw_model.extract_keywords(documents, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)
  return keywords
# model_100langs = BertModel.from_pretrained('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
# kw_model_100langs = KeyBERT(model_100langs)
# keywords_100langs = kw_model_100langs.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)


In [ ]:
# gpu 사용법
# https://github.com/MaartenGr/KeyBERT/issues/108
# model = SentenceTransformer(
#     "<hf_model_name>",
#     device="cuda:0"
# )
model = BertModel.from_pretrained('skt/kobert-base-v1')
model = model.cuda()
print("DEVICE", model.device)
# hf_model = KeyBERT(model)

# https://www.kaggle.com/code/accountstatus/shopee-competition-using-bert-model


In [ ]:
# print(os.getcwd())
# base_path = "/content/drive/MyDrive/쳇, 6pt: 비스킷(biskuit)/dataset/content/"
base_path = '../../../data/content/'

In [ ]:
def extract_keyword(base_path:str):
  content_dir_list = os.scandir(base_path)
  documents = []
  for content_dir in content_dir_list:
      if not content_dir.is_dir(): continue
      content_list = os.scandir(content_dir)
      for content in content_list:
          if not content.is_file(): continue
          file = open(content.path, "r")
          if len(text.split('\n')) > 50:
            summary = summarize(text)
          else:
            summary = text
          # nouns = noun_extractor(text)
          nouns = noun_extractor(summary)
          text = ' '.join(nouns)
          documents.append(text)
  # print(text)
  keyword_extractor('skt/kobert-base-v1', documents)
  # keyword_extractor('paraphrase-multilingual-MiniLM-L12-v2', documents)   # multilingual!
  # keyword_extractor('paraphrase-multilingual-mpnet-base-v2', documents)   # 시간 더 오래걸리고 결과 더 조음!

In [ ]:

faster_keywords = extract_keyword(base_path)
faster_counter = Counter(faster_keywords)
print(faster_counter.most_common(100))

In [ ]:
# file = open(data_path + "2022-09-22_29CM_예약하기 서비스 개발기 (1)" + ".txt", "r")
content_dir_list = os.scandir(base_path)
keywords = [] 
for content_dir in tqdm(content_dir_list):
  if not content_dir.is_dir(): continue
  content_list = os.scandir(content_dir)
  for content in content_list:
    if not content.is_file(): continue
    file = open(content.path, "r")
    keywords.append(extract_keyword(file))
counter = Counter(keywords)
print(counter.most_common(100))